# Разные задачи и инструменты по обработке текста

**Запись находится по [ссылке](https://drive.google.com/file/d/1Xftnax5whX7nDSdeim1uX48rsaoLx5Ct/view?usp=drive_link).**

In [ ]:
!pip install stanza spacy textblob yake natasha keybert dostoevsky langdetect autocorrect -q
!python -m spacy download ru_core_news_sm -q
!python -m spacy download en_core_web_sm -q
!python -m textblob.download_corpora -q
!python -m dostoevsky download fasttext-social-network-model

In [ ]:
import stanza

stanza.download('ru')
stanza.download('en')

INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: ru (Russian) ...


INFO:stanza:Downloaded file to /root/stanza_resources/ru/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Downloading default packages for language: en (English) ...


INFO:stanza:Downloaded file to /root/stanza_resources/en/default.zip
INFO:stanza:Finished downloading models and saved to /root/stanza_resources


## План
- Извлечение именованых сущностей
- Выделение именных групп
- Анализ тональности
- Выделение ключевых слов
- Исправление опечаток

Russian text:
> Мы уже показывали один день на European Conference on Computer Vision глазами Сергея Кастрюлина из Yandex Research и делились трендами и статьями, которые собрала для вас Дарья Виноградова. Ещё больше интересного привёз для вас Александр Устюжанин, разработчик в команде YandexART.

English text:
> Researchers from the AI and Applied AI centers joined the  2024 in Vienna, one of the three premier A* international conferences on machine learning and artificial intelligence, alongside NeurIPS and ICLR. On July 23, Assistant Professor Ekaterina Muravleva and Professor Ivan Oseledets presented a  titled “Neural Operators Meet Conjugate Gradients: The FCG-NO Method for Efficient PDE Solving”, introducing the FCG-NO method that leverages neural operators to enhance the accuracy of differential equation solutions.

In [ ]:
ru_text = '''Мы уже показывали один день на European Conference on Computer Vision глазами Сергея Кастрюлина из Yandex Research и делились трендами и статьями, которые собрала для вас Дарья Виноградова. Ещё больше интересного привёз для вас Александр Устюжанин, разработчик в команде YandexART.'''

In [ ]:
en_text = '''Researchers from the AI and Applied AI centers joined the  2024 in Vienna, one of the three premier A* international conferences on machine learning and artificial intelligence, alongside NeurIPS and ICLR. On July 23, Assistant Professor Ekaterina Muravleva and Professor Ivan Oseledets presented a  titled “Neural Operators Meet Conjugate Gradients: The FCG-NO Method for Efficient PDE Solving”, introducing the FCG-NO method that leverages neural operators to enhance the accuracy of differential equation solutions. '''

## Извлечение именованных сущностей (Named-entity recognition, NER)

+ Задача NER – выделить спаны именованных сущностей в тексте
+ Изначально, именованные сущности – это персоны, локации, организации
+ Обычно типов больше: даты, денежные суммы, прочее (например, названия брендов)
+ Зачем? Решать задачи референции, референциального выбора и кореференции, метонимии, которые являются центральными для поиска, вопросно-ответных систем, связности текста, синтаксического и морфологического парсинга и т.д.
+ Сложности:
    - омонимия: "Вашингтон" – город, штат, фамилия, имя жирафа, название компании?
    - технические: какие теги? где границы сущности?
+ BIOES-схема: к метке сущности (например, PER для персон или ORG для организаций) добавляется префикс, который обозначает позицию токена в спане сущности:

  - B – beginning – первый токен в спане сущности, которая состоит из нескольких токенов;
  - I – inside – внутри спана;
  - О – outside – токен не относится ни к какой сущности;
  - E – ending – последний токен сущности, которая состоит из нескольких токенов;
  - S – single – сущность состоит из одного токена.

| Иванов | Иван | Иванович | ушел |
| -- | -- | -- | -- |
| B-PER | I-PER | E-PER | O |

### Stanza

+ https://stanfordnlp.github.io/stanza/
+ библиотека для NLP
+ 66 языков, включая русский
+ токенизация, лемматизация, морфологический и синтаксический парсинг, NER

Для выявления именованных сущностей необходимо токенизировать текст.

In [ ]:
nlp = stanza.Pipeline(lang='ru', processors='tokenize,ner')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: ru (Russian):
| Processor | Package   |
-------------------------
| tokenize  | syntagrus |
| ner       | wikiner   |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user vi

In [ ]:
doc = nlp(ru_text)

print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: European Conference on Computer Vision	type: MISC
entity: Сергея Кастрюлина	type: PER
entity: Yandex Research	type: ORG
entity: Дарья Виноградова	type: PER
entity: Александр Устюжанин	type: PER
entity: YandexART	type: MISC


Если нужны BIOES NER теги для каждого токена:

In [ ]:
print(*[f'token: {token.text}\tner: {token.ner}' for sent in doc.sentences for token in sent.tokens][10:50], sep='\n')

token: Vision	ner: E-MISC
token: глазами	ner: O
token: Сергея	ner: B-PER
token: Кастрюлина	ner: E-PER
token: из	ner: O
token: Yandex	ner: B-ORG
token: Research	ner: E-ORG
token: и	ner: O
token: делились	ner: O
token: трендами	ner: O
token: и	ner: O
token: статьями	ner: O
token: ,	ner: O
token: которые	ner: O
token: собрала	ner: O
token: для	ner: O
token: вас	ner: O
token: Дарья	ner: B-PER
token: Виноградова	ner: E-PER
token: .	ner: O
token: Ещё	ner: O
token: больше	ner: O
token: интересного	ner: O
token: привёз	ner: O
token: для	ner: O
token: вас	ner: O
token: Александр	ner: B-PER
token: Устюжанин	ner: E-PER
token: ,	ner: O
token: разработчик	ner: O
token: в	ner: O
token: команде	ner: O
token: YandexART	ner: S-MISC
token: .	ner: O


Посмотрим то же самое для английского

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,ner')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package                   |
-----------------------------------------
| tokenize  | combined                  |
| mwt       | combined                  |
| ner       | ontonotes-ww-multi_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
/usr/local/lib/python3.10/dist-packages/stanza/models/tokenization/trainer.py:82: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects 

In [ ]:
doc = nlp(en_text)
print(*[f'entity: {ent.text}\ttype: {ent.type}' for sent in doc.sentences for ent in sent.ents], sep='\n')

entity: AI and Applied AI	type: ORG
entity: 2024	type: DATE
entity: Vienna	type: GPE
entity: one	type: CARDINAL
entity: three	type: CARDINAL
entity: A*	type: ORG
entity: NeurIPS	type: ORG
entity: ICLR	type: ORG
entity: July 23	type: DATE
entity: Ekaterina Muravleva	type: PERSON
entity: Ivan Oseledets	type: PERSON
entity: FCG	type: ORG


[Вот тут](https://stanfordnlp.github.io/stanza/performance.html) можно посмотреть, какие языки доступны в библиотеке по умолчанию, а какие надо скачивать. Также там можно увидеть качество на различных языках

### SpaCy

+ Библиотека для продвинутого NLP
+ Ряд языков, английский, китайский, немецкий, французский, итальянский, польский, испанский и др., разрабатываются модели для всё новых языков
+ Про spaCy: https://spacy.io/usage



In [ ]:
import spacy
from spacy.tokens import Span
from spacy import displacy

In [ ]:
nlp = spacy.load("ru_core_news_sm")

In [ ]:
doc = nlp(ru_text)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

Сергея Кастрюлина 78 95 PER
Yandex Research 99 114 ORG
Дарья Виноградова 171 188 PER
Александр Устюжанин 228 247 PER
YandexART 271 280 ORG


In [ ]:
displacy.render(doc, style = "ent", jupyter=True)

SpaCy предлагает 18 тегов (список можно посмотреть [здесь](https://towardsdatascience.com/named-entity-recognition-ner-using-spacy-nlp-part-4-28da2ece57c6)) для разных типов именованных сущностей, также легко можно добавить новый тег:

In [ ]:
nlp = spacy.load("en_core_web_sm")
doc = nlp(en_text)
ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
ents

[('AI', 21, 23, 'ORG'),
 ('Applied AI', 28, 38, 'ORG'),
 ('2024', 59, 63, 'DATE'),
 ('Vienna', 67, 73, 'GPE'),
 ('one', 75, 78, 'CARDINAL'),
 ('three', 86, 91, 'CARDINAL'),
 ('ICLR', 200, 204, 'ORG'),
 ('July 23', 209, 216, 'DATE'),
 ('Ekaterina Muravleva', 238, 257, 'PERSON'),
 ('Ivan Oseledets', 272, 286, 'PERSON'),
 ('Neural Operators Meet Conjugate Gradients: The', 308, 354, 'WORK_OF_ART'),
 ('FCG', 413, 416, 'ORG')]

У нас здесь снова ошибка в длинном названии статьи, давайте поправим и нарисуем результат

In [ ]:
ns2_ent = Span(doc, 53, 68, label="WORK_OF_ART")
doc.ents = list(doc.ents[:-2]) + [ns2_ent]

ents = [(e.text, e.start_char, e.end_char, e.label_) for e in doc.ents]
ents

[('AI', 21, 23, 'ORG'),
 ('Applied AI', 28, 38, 'ORG'),
 ('2024', 59, 63, 'DATE'),
 ('Vienna', 67, 73, 'GPE'),
 ('one', 75, 78, 'CARDINAL'),
 ('three', 86, 91, 'CARDINAL'),
 ('ICLR', 200, 204, 'ORG'),
 ('July 23', 209, 216, 'DATE'),
 ('Ekaterina Muravleva', 238, 257, 'PERSON'),
 ('Ivan Oseledets', 272, 286, 'PERSON'),
 ('Neural Operators Meet Conjugate Gradients: The FCG-NO Method for Efficient PDE Solving',
  308,
  394,
  'WORK_OF_ART')]

In [ ]:
displacy.render(doc, style = "ent", jupyter=True)

Точно так же мы можем исправить и остальные ошибки

Можно подкрасить только сущности определенных типов:

In [ ]:
options = {'ents': ['ORG', 'LOC']}
displacy.render(doc, style = "ent", jupyter=True, options=options)

[Вот тут](https://spacy.io/models) можно узнать, какие языки доступны и как называется моджель для вашего языка

### Natasha

+ Раньше библиотека Natasha решала задачу NER для русского языка, была построена на правилах
+ сейчас это полноценный NLP проект для русского языка
+ токенизация, лемматизация, синтаксический разбор, NER-тегирование и т.д.
+ https://github.com/natasha/natasha

In [ ]:
from natasha import (
    Segmenter,
    MorphVocab,

    NewsEmbedding,
    NewsMorphTagger,
    NewsSyntaxParser,
    NewsNERTagger,

    PER,
    NamesExtractor,

    Doc
)

In [ ]:
segmenter = Segmenter()
morph_vocab = MorphVocab()

emb = NewsEmbedding()
morph_tagger = NewsMorphTagger(emb)
syntax_parser = NewsSyntaxParser(emb)
ner_tagger = NewsNERTagger(emb)

names_extractor = NamesExtractor(morph_vocab)
doc = Doc(ru_text)

NER зависит от сегментации:

In [ ]:
doc.segment(segmenter)
print(doc.tokens[:5])
print(doc.sents[:5])

[DocToken(stop=2, text='Мы'), DocToken(start=3, stop=6, text='уже'), DocToken(start=7, stop=17, text='показывали'), DocToken(start=18, stop=22, text='один'), DocToken(start=23, stop=27, text='день')]
[DocSent(stop=189, text='Мы уже показывали один день на European Conferenc..., tokens=[...]), DocSent(start=190, stop=281, text='Ещё больше интересного привёз для вас Александр У..., tokens=[...])]


In [ ]:
doc.tag_ner(ner_tagger)
display(doc.spans)

[DocSpan(start=78, stop=95, type='PER', text='Сергея Кастрюлина', tokens=[...]),
 DocSpan(start=99, stop=114, type='ORG', text='Yandex Research', tokens=[...]),
 DocSpan(start=171, stop=188, type='PER', text='Дарья Виноградова', tokens=[...]),
 DocSpan(start=228, stop=247, type='PER', text='Александр Устюжанин', tokens=[...]),
 DocSpan(start=271, stop=280, type='ORG', text='YandexART', tokens=[...])]

In [ ]:
doc.ner.print()

Мы уже показывали один день на European Conference on Computer Vision 
глазами Сергея Кастрюлина из Yandex Research и делились трендами и 
        PER──────────────    ORG────────────                       
статьями, которые собрала для вас Дарья Виноградова. Ещё больше 
                                  PER──────────────             
интересного привёз для вас Александр Устюжанин, разработчик в команде 
                           PER────────────────                        
YandexART.
ORG────── 


Можно привести сущности к нормальной форме, для этого надо провести морфологический анализ и лемматизацию (Natasha использует Pymorphy2):

In [ ]:
doc.tag_morph(morph_tagger)
doc.sents[0].morph.print()

                  Мы PRON|Case=Nom|Number=Plur|Person=1
                 уже ADV|Degree=Pos
          показывали VERB|Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Act
                один NUM|Animacy=Inan|Case=Acc|Gender=Masc
                день NOUN|Animacy=Inan|Case=Acc|Gender=Masc|Number=Sing
                  на ADP
            European X|Foreign=Yes
          Conference X|Foreign=Yes
                  on X|Foreign=Yes
            Computer X|Foreign=Yes
              Vision X|Foreign=Yes
             глазами NOUN|Animacy=Inan|Case=Ins|Gender=Masc|Number=Plur
              Сергея PROPN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
          Кастрюлина PROPN|Animacy=Anim|Case=Gen|Gender=Masc|Number=Sing
                  из ADP
              Yandex X|Foreign=Yes
            Research X|Foreign=Yes
                   и CCONJ
            делились VERB|Aspect=Imp|Mood=Ind|Number=Plur|Tense=Past|VerbForm=Fin|Voice=Mid
            трендами NOUN|Animacy=Inan|Case=Ins|Gende

Приводим сущности к нормальной форме:

In [ ]:
for span in doc.spans:
  span.normalize(morph_vocab)

{_.text: _.normal for _ in doc.spans}

{'Сергея Кастрюлина': 'Сергей Кастрюлин',
 'Yandex Research': 'Yandex Research',
 'Дарья Виноградова': 'Дарья Виноградова',
 'Александр Устюжанин': 'Александр Устюжанин',
 'YandexART': 'YandexART'}

Можно извлечь для нормированных имен отдельно имена и фамилии:

In [ ]:
for span in doc.spans:
  if span.type == PER:
    span.extract_fact(names_extractor)

{_.normal: _.fact.as_dict for _ in doc.spans if _.type == PER}

{'Сергей Кастрюлин': {'first': 'Сергей', 'last': 'Кастрюлин'},
 'Дарья Виноградова': {'first': 'Дарья', 'last': 'Виноградова'},
 'Александр Устюжанин': {'first': 'Александр', 'last': 'Устюжанин'}}

## Выделение именных групп

### SpaCy

В SpaCy есть возможность извлекать именные группы из синтаксического разбора одной строчкой, однако это получается не для всех языков. Доя русского, например, пока такая функуия не реализована:

In [ ]:
nlp = spacy.load("ru_core_news_sm")

doc = nlp(ru_text)

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text, sep='\t')

NotImplementedError: [E894] The 'noun_chunks' syntax iterator is not implemented for language 'ru'.

Но можно так делать для английского:

In [ ]:
nlp = spacy.load("en_core_web_sm")

doc = nlp(en_text)

for chunk in doc.noun_chunks:
    print(chunk.text, chunk.root.text, chunk.root.dep_,
            chunk.root.head.text, sep='\t')

Researchers	Researchers	nsubj	joined
the AI and Applied AI centers	centers	pobj	from
Vienna	Vienna	pobj	in
the three premier A* international conferences	conferences	pobj	of
machine learning	learning	pobj	on
artificial intelligence	intelligence	conj	learning
NeurIPS	NeurIPS	pobj	alongside
ICLR	ICLR	conj	NeurIPS
July	July	pobj	On
Assistant Professor Ekaterina Muravleva	Muravleva	nsubj	presented
Professor Ivan Oseledets	Oseledets	conj	Muravleva
a  titled “Neural Operators Meet Conjugate Gradients	Gradients	dobj	presented
The FCG-NO Method	Method	appos	Gradients
Efficient PDE Solving	Solving	pobj	for
the FCG-NO method	method	dobj	introducing
that	that	nsubj	leverages
neural operators	operators	dobj	leverages
the accuracy	accuracy	dobj	enhance
differential equation solutions	solutions	pobj	of


### TextBlob

Еще для множества задач для английского можно испольщовать TextBlob. Для извлечения именных групп в том числе

Дока: https://textblob.readthedocs.io/en/dev/index.html

In [ ]:
import nltk
from textblob import TextBlob

nltk.download('punkt_tab')

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
blob = TextBlob(en_text)
blob.noun_phrases

WordList(['researchers', 'ai', 'applied ai', 'vienna', 'international conferences', 'machine learning', 'artificial intelligence', 'neurips', 'iclr', 'july', 'professor ekaterina muravleva', 'professor ivan oseledets', 'neural operators', 'conjugate gradients', 'fcg-no method', 'efficient pde solving', 'fcg-no', 'leverages neural operators', 'differential equation solutions'])

## Извлечение ключевых слов

### YAKE

- Дока: https://github.com/LIAAD/yake
- Работает на основе статистики, поэтому быстрее своих нейросетевых аналогов, но чаще ошибается

In [ ]:
from yake import KeywordExtractor

In [ ]:
max_ngram_size = 2
num_words = 5

In [ ]:
language = 'ru'
custom_kw_extractor = KeywordExtractor(lan=language, n=max_ngram_size, top=num_words)
keywords = custom_kw_extractor.extract_keywords(ru_text)

for kw in keywords:
    print(kw)

('Дарья Виноградова', 0.00643573886924208)
('European Conference', 0.00950861230958936)
('Computer Vision', 0.00950861230958936)
('Сергея Кастрюлина', 0.00950861230958936)
('Yandex Research', 0.00950861230958936)


In [ ]:
language = 'en'
custom_kw_extractor = KeywordExtractor(lan=language, n=max_ngram_size, top=num_words)
keywords = custom_kw_extractor.extract_keywords(en_text)

for kw in keywords:
    print(kw)

('international conferences', 0.01205398821157259)
('artificial intelligence', 0.01205398821157259)
('alongside NeurIPS', 0.01205398821157259)
('centers joined', 0.015340072684237148)
('machine learning', 0.015340072684237148)


### KeyBERT

- Дока: https://maartengr.github.io/KeyBERT/api/keybert.html
- Нейросеть, поэтому работает дольше и больше весит, но дает выше качество

In [ ]:
from keybert import KeyBERT

При инициализации можно использовать дюбую модель, которая умеет выдавать эмбеддинги. Собственно, он нее и будет зависеть качество работы

In [ ]:
model = KeyBERT('DeepPavlov/rubert-base-cased') # неплохой берт для русского

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/642 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/714M [00:00<?, ?B/s]

Some weights of the model checkpoint at DeepPavlov/rubert-base-cased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tokenizer_config.json:   0%|          | 0.00/24.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/1.65M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
keywords = model.extract_keywords(ru_text, keyphrase_ngram_range=(1, 3),
                                  top_n=5)
for keyword in keywords:
    print(keyword)

('research делились трендами', 0.4981)
('кастрюлина из yandex', 0.4795)
('ещё больше интересного', 0.4488)
('vision глазами сергея', 0.4385)
('из yandex research', 0.4378)


Посмотрим для английского:

In [ ]:
model = KeyBERT('distilbert-base-nli-mean-tokens')

modules.json:   0%|          | 0.00/229 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/122 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/4.02k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/550 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/450 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [ ]:
keywords = model.extract_keywords(en_text, keyphrase_ngram_range=(1, 3),
                                  top_n=5)
for keyword in keywords:
    print(keyword)

('2024 vienna premier', 0.6677)
('joined 2024 vienna', 0.6337)
('2024 vienna', 0.578)
('centers joined 2024', 0.5193)
('vienna premier international', 0.4505)


## Анализ тональности текста

Анализ тональности текста $-$ это задача определения настроения текста, обычно положительный / отрицательный / нейтральный, но есть варианты, где распознаются эмоции, как в text2emotion.

Для русского есть библиотека dostoevsky.

### dostoevsky

In [ ]:
from dostoevsky.tokenization import RegexTokenizer
from dostoevsky.models import FastTextSocialNetworkModel

In [ ]:
tokenizer = RegexTokenizer()
model = FastTextSocialNetworkModel(tokenizer=tokenizer)

In [ ]:
messages = [
    "всё довольно неплохо",
    "здесь нет ничего хорошего",
    "всё очень плохо",
    "просто ужасно",
    "замечательно сказано"
]

results = model.predict(messages, k=2)

for message, sentiment in zip(messages, results):
    print(message, '->', sentiment)

всё довольно неплохо -> {'positive': 0.9777238368988037, 'negative': 0.4378334879875183}
здесь нет ничего хорошего -> {'neutral': 0.6442351341247559, 'negative': 0.08270734548568726}
всё очень плохо -> {'negative': 0.7606606483459473, 'positive': 0.5000100135803223}
просто ужасно -> {'negative': 0.9814634323120117, 'skip': 0.053413331508636475}
замечательно сказано -> {'positive': 0.9553291201591492, 'skip': 0.031153826043009758}


А для английского так умеют Stanza и TextBlob

### Stanza

In [ ]:
nlp = stanza.Pipeline(lang='en', processors='tokenize,sentiment')

INFO:stanza:Checking for updates to resources.json in case models have been updated.  Note: this behavior can be turned off with download_method=None or download_method=DownloadMethod.REUSE_RESOURCES


INFO:stanza:Downloaded file to /root/stanza_resources/resources.json
INFO:stanza:Loading these models for language: en (English):
| Processor | Package        |
------------------------------
| tokenize  | combined       |
| mwt       | combined       |
| sentiment | sstplus_charlm |

INFO:stanza:Using device: cpu
INFO:stanza:Loading: tokenize
INFO:stanza:Loading: mwt
INFO:stanza:Loading: sentiment
INFO:stanza:Done loading processors!


In [ ]:
# The existing models each support negative, neutral, and positive,
# represented by 0, 1, 2 respectively.

en_messages = [
    "everything is pretty good",
    "there is nothing good here",
    "everything is very bad",
    "just terrible",
    "well said",
    "cats are animals"
]
doc = nlp('.\n'.join(en_messages))
for i, sentence in enumerate(doc.sentences):
    print(sentence.text, "%d -> %d" % (i, sentence.sentiment))

everything is pretty good. 0 -> 2
there is nothing good here. 1 -> 0
everything is very bad. 2 -> 0
just terrible. 3 -> 0
well said. 4 -> 1
cats are animals 5 -> 1


### TextBlob

In [ ]:
# Return a tuple of form (polarity, subjectivity ) where polarity is a float
# within the range [-1.0, 1.0] and subjectivity is a float within
# the range [0.0, 1.0] where 0.0 is very objective and 1.0 is very subjective.
for message in en_messages:
    blob = TextBlob(message)
    print(message, '\t', blob.sentiment)

everything is pretty good 	 Sentiment(polarity=0.475, subjectivity=0.8)
there is nothing good here 	 Sentiment(polarity=0.7, subjectivity=0.6000000000000001)
everything is very bad 	 Sentiment(polarity=-0.9099999999999998, subjectivity=0.8666666666666667)
just terrible 	 Sentiment(polarity=-1.0, subjectivity=1.0)
well said 	 Sentiment(polarity=0.0, subjectivity=0.0)
cats are animals 	 Sentiment(polarity=0.0, subjectivity=0.0)


## Определение языка

При работе с большими данными часто нужно разделить данные на группы по языкам, чтобы отправить на обработку в разные модели, например. Для этого есть разные инструменты, например, основанные на простом распределении частотности букв, на словарях, с нейросетевым подходом.

In [ ]:
from langdetect import detect, detect_langs
from langdetect import DetectorFactory

DetectorFactory.seed = 0

In [ ]:
# Dschinghis Khan - Moskau
detect("Moskau — Tor zur Vergangenheit, Spiegel der Zarenzeit, Rot wie das Blut")

'de'

In [ ]:
detect_langs("Moskau — Tor zur Vergangenheit, Spiegel der Zarenzeit, Rot wie das Blut")

[de:0.9999953675874703]

Часто бывает так, что на коротких текстах библиотека ошибается и выдает какие-то другие языки, чаще всего родственные или со схожим алфавитом.

In [ ]:
detect_langs("Привет!")

[mk:0.8192481473701985, bg:0.18075185259138762]

In [ ]:
detect_langs("Привет, ты как?")

[ru:0.9999985230201999]

In [ ]:
# Ricchi E Poveri - Voulez Vous Danser

detect_langs("""
Questa musica è un'isola in mezzo al mare
basta chiudere gli occhi e saprai dovè
devi solo sapertele conquistare
voulez-vous voulez-vous voulez-vous danser?""")

[it:0.9999940963128858]

Бывает внезапное

In [ ]:
detect_langs("Voulez-vous, voulez-vous, voulez-vous danser?")

[cs:0.8571396370596909, fr:0.14285596647041326]

In [ ]:
detect_langs("Voulez-vous danser?")

[fr:0.5714290893675289, nl:0.42857062095631704]

Можно сделать свою модель, например, по википедии посчитав распределение букв в разных языках и выдавая ближайший вектор распредлеения

## Спеллчекер / автокоррект

Автоматическое исправление опечаток - это большая задача. Обычно этим занимаются компании, работающие с текстовым вводом:

- Дока: https://github.com/filyp/autocorrect
- редакторы документов (Word, Google Docs)
- различные исправляющие сервисы (Grammarly)
- клавиатуры для мобильных устройств (Google, Yandex, любые другие)

In [ ]:
from autocorrect import Speller

Для английского:

In [ ]:
spell = Speller(lang='en')

In [ ]:
spell("I'm not sleapy and tehre is no place I'm giong to.")

"I'm not sleepy and there is no place I'm going to."

In [ ]:
%timeit spell("There is no comin to consiousnes without pain.")

174 ms ± 42.6 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


Можно ускорить работу до скорости меньше миллисекунды.

In [ ]:
spell = Speller(fast=True)

In [ ]:
%timeit spell("There is no comin to consiousnes without pain.")

573 µs ± 123 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


Для русского:

In [ ]:
spell = Speller('ru')

In [ ]:
spell("Опечатак - это енприятно")

'Опечатка - это неприятно'